In [ ]:
import psycopg2
import geopandas as gpd
import os
import sys
import glob
import datetime
import zipfile

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
    
from wp4.constants import DB_HOST, DB_NAME, DB_USER, DB_PASS, DATA_DIR_FIRES

## Unzip the directories containing the shapefiles, if this has not already been done

In [ ]:
for file in glob.glob(f'{DATA_DIR_FIRES}/*.zip'):
    if not os.path.exists(file.replace('.zip', '')):
        os.makedirs(file.replace('.zip', ''))
        
    with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(file.replace('.zip', ''))
            
    os.remove(file)

# Initiate connection to spatial database (PostgreSQL/PostGIS)

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

### Create a new table for the storage of fire event information 

In [ ]:
cur.execute(
    """
    CREATE TABLE all_fire_events_2(
    ID SERIAL PRIMARY KEY,
    DATETIME timestamp,
    GEOMETRY geometry,
    SOURCE varchar(200),
    LOCATION varchar(500),
    REFERENCE varchar(500),
    TYPE varchar(500),
    INFO varchar(1000),
    FRP NUMERIC(10,5)
    )"""
    )

conn.commit()

### Create a dictionary with all the information needed to combine the information from different data sources into a single table 

In [ ]:
column_info = {
    'DATETIME':{ # Column names containing the Date and/or Time information
        'BlackstairsMountains_Brigade_Database':{
            'DATE':'Date',
            'TIME':'From_',
            'PATTERN':'%Y-%m-%d%H%M', # pattern to obtain a datetime object 
        },
        'Clare_Brigade_Database':{
            'DATE':'Date_and_T',
            'PATTERN':'%Y-%m-%d',
        },
        'Leinster_Database':{
            'DATE':'Date',
            'PATTERN':'%Y-%m-%d',
        },
        'Leitrim_Brigade_Database':{
            'DATE':'Assign_Tim',
            'PATTERN':'%Y-%m-%d',
        },
        'Mayo_Brigade_Database':{
            'DATE':'Assign_Tim',
            'PATTERN':'%Y-%m-%d',
        },
        'Meath_Brigade_Database':{
            'DATE':'Received',
            'PATTERN':'%Y-%m-%d',
        },
        'Media_Reports_Database':{
            'DATE':'Date',
            'PATTERN':'%Y-%m-%d',
        },
        'Roscommon_Brigade_Database':{
            'DATE':'Assign_Tim',
            'PATTERN':'%Y-%m-%d',
        },
        'Satellite_hotspots_Database':{
            'DATE':'ACQ_DATE',
            'TIME':'ACQ_TIME',
            'PATTERN':'%Y-%m-%d%H%M',
        },
        'WestCork_Brigade_Database':{
            'DATE':'Date',
            'PATTERN':'%Y-%m-%d',
        },
        'Westmeath_Brigade_Database':{
            'DATE':'Date',
            'PATTERN':'%Y-%m-%d',
        },
        'Wicklow_Brigade':{
            'DATE':'Date',
            'PATTERN':'%Y-%m-%d',
        },     
    },
    'SOURCE':{  # To use as property indicating the general source of the data (Fire Brigade, Satellite or Media)
        'BlackstairsMountains_Brigade_Database':'Fire Brigade',
        'Clare_Brigade_Database':'Fire Brigade',
        'Leinster_Database':'Fire Brigade',
        'Leitrim_Brigade_Database':'Fire Brigade',
        'Mayo_Brigade_Database':'Fire Brigade',
        'Meath_Brigade_Database':'Fire Brigade',
        'Media_Reports_Database':'Media',
        'Roscommon_Brigade_Database':'Fire Brigade',
        'Satellite_hotspots_Database':'Satellite',
        'WestCork_Brigade_Database':'Fire Brigade',
        'Westmeath_Brigade_Database':'Fire Brigade',
        'Wicklow_Brigade':'Fire Brigade',
    },
    'LOCATION':{ # Column containing the location information
        'BlackstairsMountains_Brigade_Database':'Townland',
        'Clare_Brigade_Database':'Town_Area',
        'Leinster_Database':'DISTRICT_I',
        'Leitrim_Brigade_Database':'Add_Line_5',
        'Mayo_Brigade_Database':'Add_Line_5',
        'Meath_Brigade_Database':'District',
        'Media_Reports_Database':'Location',
        'Roscommon_Brigade_Database':'Add_Line_5',
        'Satellite_hotspots_Database':None,
        'WestCork_Brigade_Database':'Townland',
        'Westmeath_Brigade_Database':'DISTRICT_I',
        'Wicklow_Brigade':'Location1',
    },
    'REFERENCE':{  # Reference to the specific source of information
        'BlackstairsMountains_Brigade_Database':'Blackstairs Mountains Fire Brigade',
        'Clare_Brigade_Database':'Clare Fire Brigade',
        'Leinster_Database':'Leinster Fire Brigade',
        'Leitrim_Brigade_Database':'Leitrim Fire Brigade',
        'Mayo_Brigade_Database':'Mayo Fire Brigade',
        'Meath_Brigade_Database':'Meath Fire Brigade',
        'Media_Reports_Database':'Reference',
        'Roscommon_Brigade_Database':'Roscommon Fire Brigade',
        'Satellite_hotspots_Database':'SATELLITE',
        'WestCork_Brigade_Database':'West Cork Fire Brigade',
        'Westmeath_Brigade_Database':'Westmeath Fire Brigade',
        'Wicklow_Brigade':'Wicklow Fire Brigade',
    },
    'INFO':{  # In case detailed information about the fire is available 
        'BlackstairsMountains_Brigade_Database':None,
        'Clare_Brigade_Database':'Class_Desc',
        'Leinster_Database':None,
        'Leitrim_Brigade_Database':'Class_Desc',
        'Mayo_Brigade_Database':'Class_Desc',
        'Meath_Brigade_Database':None,
        'Media_Reports_Database':None,
        'Roscommon_Brigade_Database':'Class_Desc',
        'Satellite_hotspots_Database':None,
        'WestCork_Brigade_Database':None,
        'Westmeath_Brigade_Database':'Class_Desc',
        'Wicklow_Brigade':None,
    },
    'FRP':{  # In case detailed information about the fire is available 
        'BlackstairsMountains_Brigade_Database':None,
        'Clare_Brigade_Database':None,
        'Leinster_Database':None,
        'Leitrim_Brigade_Database':None,
        'Mayo_Brigade_Database':None,
        'Meath_Brigade_Database':None,
        'Media_Reports_Database':None,
        'Roscommon_Brigade_Database':None,
        'Satellite_hotspots_Database':'FRP',
        'WestCork_Brigade_Database':None,
        'Westmeath_Brigade_Database':None,
        'Wicklow_Brigade':None,
    },
    'TYPE':{ # Collection of dictionaries to use to reassign labels specifying the type of Fire
        # The following labels been used: Bog Fire, Bush Fire, Grass Fire, Forest Fire, Gorse Fire, Controlled Fire
        'BlackstairsMountains_Brigade_Database':None,
        'Clare_Brigade_Database':{
            'Incident_T' :{
                'FIRE BOG':'Bog Fire',
                'FIRE BUSH':'Bush Fire',
                'FIRE FIELD':'Grass Fire',
                'FIRE FOREST':'Forest Fire',
                'FIRE GARAGE':None,
                'FIRE GORSE':'Gorse Fire',
                'FIRE GRASS':'Grass Fire',
                'FIRE OUTDOOR':None,
            }
        },
        'Leinster_Database':{
            'Descriptio' :{
                'FIRE/CONTROLLED BURNING':'Controlled Fire',
                'FIRE/GORSE':'Gorse Fire',
                'FIRE/GRASS':'Grass Fire',
                'FIRE/SMOKE ISSUING':None,
                'FIRE/FOREST':'Forest Fire',
            }
        },
        'Leitrim_Brigade_Database' :{
            'Revised_In':{
                'GRASS FIRE':'Grass Fire',
                'FOREST FIRE':'Forest Fire',
                'GORSE FIRE':'Gorse Fire',
                'BOG FIRE':'Bog Fire', 
            }
        },
        'Mayo_Brigade_Database' :{
            'Revised_In':{
                'GRASS FIRE':'Grass Fire',
                'FOREST FIRE':'Forest Fire',
                'GORSE FIRE':'Gorse Fire',
                'BOG FIRE':'Bog Fire', 
            }
        },
        'Meath_Brigade_Database' :{
            'Desc_':{
                'FIRE/GRASS':'Grass Fire',
                'FIRE/GORSE':'Gorse Fire',
                'FIRE/FOREST':'Forest Fire',
            }
        },
        'Media_Reports_Database':None,
        'Roscommon_Brigade_Database':{
            'Revised_In':{
                'GRASS FIRE':'Grass Fire',
                'FOREST FIRE':'Forest Fire',
                'GORSE FIRE':'Gorse Fire',
                'BOG FIRE':'Bog Fire', 
            }
        },
        'Satellite_hotspots_Database':None,
        'WestCork_Brigade_Database':None,
        'Westmeath_Brigade_Database':{
            'Descriptio' :{
                'FIRE/CONTROLLED BURNING':'Controlled Fire',
                'FIRE/GORSE':'Gorse Fire',
                'FIRE/GRASS':'Grass Fire',
                'FIRE/SMOKE ISSUING':None,
                'FIRE/FOREST':'Forest Fire',
            }
        },
        'Wicklow_Brigade':{
            'Label' :{
                'Fire Gorse':'Gorse Fire',
                'Fire Forest':'Forest Fire',
                'Fire Grass':'Grass Fire',
                'Fire gorse':'Gorse Fire',
                '_':None,
                'Hill on fire':None,
                'Bog/gorse on fire':'Bog Fire',
                'Bog / gorse on fire':'Bog Fire',
                'Bushes on fire':'Bush Fire',
                'Gorse on fire':'Gorse Fire',
                'Trees/forest fire':'Forest Fire',
                'Trees / forest fire':'Forest Fire',
                'Grass on fire':'Grass Fire',
                'Gorse on fire in quarry':'Gorse Fire',
                'National park':None,
                'Small trees on fire':'Forest Fire',
                'Bushes of fire':'Bush Fire',
                'Trees on fire':'Forest Fire',
            }
        },
    }
}

In [ ]:
# List all the subdirectories in the data directory
datasets = os.listdir(DATA_DIR_FIRES)


for dataset_name in datasets:  # loop through the directories
    
    # Load all the column information for the specific dataset    
    time_info = column_info['DATETIME'][dataset_name]
    label_info = column_info['TYPE'][dataset_name] 
    source_info = column_info['SOURCE'][dataset_name]
    location_info = column_info['LOCATION'][dataset_name]
    reference_info = column_info['REFERENCE'][dataset_name]
    info_info = column_info['INFO'][dataset_name]
    frp_info = column_info['FRP'][dataset_name]
    
    # Load the shapefile, drop rows with null values for the geometry/date column and reproject to WGS 84
    dataset = gpd.read_file(glob.glob(f"{DATA_DIR_FIRES}/{dataset_name}/*.shp")[0])
    dataset = dataset.dropna(subset=['geometry', time_info['DATE']]).copy()
    
    def convert_to_datetime(x):
        """Function to convert the date and/or time information into a datetime object"""

        if 'TIME' in time_info.keys():
            # For some datasets the time column was not consistently formatted
            # as a result some try/except blocks were needed
            try:  
                return datetime.datetime.strptime(
                        f"{x[time_info['DATE']]}{x[time_info['TIME']]}",
                        time_info['PATTERN'])
            except ValueError:
                try:
                    return datetime.datetime.strptime(
                    f"{x[time_info['DATE']]}{int(x[time_info['TIME']])}",
                    time_info['PATTERN']
                )
                except:
                    return datetime.datetime.strptime(
                        f"{x[time_info['DATE']]}{x[time_info['TIME']]}",
                        '%Y-%m-%d%H:%M:%S'
                    )            
                
        else:
            return datetime.datetime.strptime(
                f"{x[time_info['DATE']]}",
                time_info['PATTERN']
        )
        
    def assign_labels(x):
        """Function to give fire types from different databases consistent naming"""
        try:
            return label_info[list(label_info.keys())[0]][x]
        except:
            return None
    
    dataset['DATETIME'] = dataset.apply(convert_to_datetime, 1)
    
    # Add columns to the dataset containing the information that will be inserted into the database
    if label_info is not None:
        dataset['TYPE'] = dataset[list(label_info.keys())[0]].apply(assign_labels)
    else:
        dataset['TYPE'] = None     
    if source_info is not None:
        dataset['SOURCE'] = source_info
    else:
        dataset['SOURCE']  = None
    if location_info is not None:
        dataset['LOCATION'] = dataset[location_info]
    else:
        dataset['LOCATION']  = None
    if reference_info is not None:
        try:
            dataset['REFERENCE'] = dataset[reference_info]
        except:
            dataset['REFERENCE'] = reference_info
    else:
        dataset['REFERENCE']  = None
    if info_info is not None:
        dataset['INFO'] = dataset[info_info]
    else:
        dataset['INFO']  = None
    
    if frp_info is not None:
        dataset['FRP'] = dataset[frp_info]
    else:
        dataset['FRP']  = None
    
    # Loop through all the fires in the dataset and insert them into the database
    for ind, row in dataset.iterrows():
        cur.execute(
        """INSERT INTO public.fire_events(
        DATETIME, GEOMETRY, SOURCE, LOCATION, REFERENCE, TYPE, INFO, FRP) VALUES(%s, ST_Transform(ST_SetSRID(ST_MakePoint(%s, %s), 2157),4326), %s, %s, %s, %s, %s, %s)""",
        (row['DATETIME'],
         row['geometry'].x,
         row['geometry'].y,
         row['SOURCE'],
         row['LOCATION'],
         row['REFERENCE'],
         row['TYPE'],
         row['INFO'],
         row['FRP']
        )
        )
        conn.commit()
